# Лабораторная работа №2: Проведение исследований с логистической и линейной регрессией

## Задача классификации

## 2. Создание бейзлайна и оценка качества

In [ ]:
X = bl_cdata.drop('HeartDisease', axis=1) # Все столбцы кроме 'HeartDisease' - это признаки
y = bl_cdata['HeartDisease'] # 'HeartDisease' - это целевая переменная, которую мы хотим предсказать

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабируем признаки, это важно для логистической регрессии
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Масштабируем обучающие данные
X_test_scaled = scaler.transform(X_test) # Масштабируем тестовые данные

# Создаем модель логистической регрессии
model = LogisticRegression(random_state=42, solver='liblinear') # solver='liblinear' для небольших наборов данных

# Обучаем модель на масштабированных обучающих данных
model.fit(X_train_scaled, y_train)

# Делаем предсказания на масштабированных тестовых данных
y_pred = model.predict(X_test_scaled)

# Вычисляем метрики
accuracy = accuracy_score(y_test, y_pred) # Доля правильных ответов
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0) # Точность предсказаний для каждого класса, взвешенная
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0) # Полнота предсказаний для каждого класса, взвешенная

# Выводим метрики
print(f"Метрики для классификации (Логистическая регрессия):")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

In [ ]:
Метрики для классификации (Логистическая регрессия):
  Accuracy: 0.8533
  Precision: 0.8572
  Recall: 0.8533

Точность для встроенных моделей sklearn оказалась очень хорошей. Попробуем улучшить бейзлайн:

## 3. Улучшение бейзлайна

Гипотеза 1: Не все признаки одинаково важны для предсказания наличия сердечных заболеваний. Отбор наиболее значимых признаков с помощью статистических методов (например, ANOVA F-статистика) может улучшить производительность модели, уменьшив шум и переобучение.
Гипотеза 2: Логистическая регрессия может быть подвержена переобучению, особенно если есть много признаков. Использование регуляризации (L1 или L2) может помочь модели обобщаться лучше, предотвращая переобучение.

In [ ]:
Отбор признаков

In [ ]:

def evaluate_model(model, X, y, cv=5):
    scoring = {'accuracy': make_scorer(accuracy_score),
                'precision': make_scorer(precision_score, average='weighted', zero_division=0),
                'recall': make_scorer(recall_score, average='weighted', zero_division=0)}
    scores = cross_validate(model, X, y, cv=cv, scoring=scoring, error_score='raise')
    return pd.DataFrame(scores).agg(['mean', 'std']).filter(regex='test_')


baseline_model = Pipeline([('scaler', StandardScaler()),
                           ('model', LogisticRegression(random_state=42, solver='liblinear'))])
baseline_scores = evaluate_model(baseline_model, X,y)
print("Бейзлайн:")
print(baseline_scores)


best_k = 0
best_score = 0
for k in range(1, X.shape[1] + 1):
    feature_selector = Pipeline([('scaler', StandardScaler()),
                                ('selector', SelectKBest(score_func=f_classif, k=k)),
                                ('model', LogisticRegression(random_state=42, solver='liblinear'))])
    scores = evaluate_model(feature_selector, X,y)
    mean_accuracy = scores['test_accuracy']['mean']
    if mean_accuracy > best_score:
        best_score = mean_accuracy
        best_k = k
print(f"\nЛучшее количество признаков: {best_k}, Accuracy: {best_score}")

In [ ]:
Бейзлайн:
      test_accuracy  test_precision  test_recall
mean       0.8267        0.8386     0.8267

Регуляризация

In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
             'penalty': ['l1', 'l2']}
grid_search = GridSearchCV(LogisticRegression(random_state=42, solver='liblinear'),
                        param_grid, cv=5, scoring='accuracy')
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
grid_search.fit(X_train_scaled, y_train)

print(f"\nЛучшие параметры регуляризации: {grid_search.best_params_}")
best_reg_model = grid_search.best_estimator_
scores = evaluate_model(Pipeline([('scaler', StandardScaler()),
                                ('model', best_reg_model)]), X,y)
print("Метрики для лучшей модели с регуляризацией:")
print(scores)

In [ ]:
      test_accuracy  test_precision  test_recall
mean       0.8278        0.8394       0.8278

In [ ]:
Предсказание на тестовой выборке и оценка качества

In [ ]:
print("\nУлучшенный бейзлайн ")
improved_baseline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(score_func=f_classif, k=best_k)),
    ('model', LogisticRegression(random_state=42, solver='liblinear', **grid_search.best_params_))
])

improved_scores = evaluate_model(improved_baseline, X, y)


improved_baseline.fit(X, y)

X_test_scaled = scaler.transform(X_test)
y_pred = improved_baseline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

print(f"Метрики для улучшенной модели (Логистическая регрессия):")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

In [ ]:
Метрики для улучшенной модели (Логистическая регрессия):
  Accuracy: 0.8696
  Precision: 0.8726
  Recall: 0.8696

С помощью отбора признаков и регуляризации получилось незначительно улучшить результаты.

## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию логистической регрессии:

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
class LogisticRegressionCustom:
    def __init__(self, learning_rate=0.01, n_iterations=1000, regularization=None, lambda_reg=0.1):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
        self.regularization = regularization
        self.lambda_reg = lambda_reg

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def _cost_function(self, y, y_predicted):
        m = len(y)
        cost = -1/m * np.sum(y * np.log(y_predicted) + (1 - y) * np.log(1 - y_predicted))
        if self.regularization == 'l1':
                cost += (self.lambda_reg / (2 * m)) * np.sum(np.abs(self.weights))
        elif self.regularization == 'l2':
            cost += (self.lambda_reg / (2 * m)) * np.sum(self.weights**2)
        return cost

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        costs = []
        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)
            cost = self._cost_function(y,y_predicted)
            costs.append(cost)
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)
            if self.regularization == 'l1':
                dw += (self.lambda_reg / n_samples) * np.sign(self.weights)
            elif self.regularization == 'l2':
                dw += (self.lambda_reg / n_samples) * self.weights

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)


Обучаем имплементированные модели и оцением их качество

In [ ]:
X = bl_cdata.drop('HeartDisease', axis=1).values
y = bl_cdata['HeartDisease'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegressionCustom(learning_rate=0.01, n_iterations=1000)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)


def evaluate_model(y_true, y_pred, model_name):
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
  recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
  print(f"Метрики для модели:")
  print(f"  Accuracy: {accuracy:.4f}")
  print(f"  Precision: {precision:.4f}")
  print(f"  Recall: {recall:.4f}")

In [ ]:
Метрики для модели:
  Accuracy: 0.8587
  Precision: 0.8634
  Recall: 0.8587

Результат даже лучше, чем у неулучшенной библиотечной модели. Теперь добавим техники из улучшенного бейзлайна

In [ ]:

feature_selector = SelectKBest(score_func=f_classif, k=best_k)
X_train_selected = feature_selector.fit_transform(X_train_scaled, y_train)
X_test_selected = feature_selector.transform(X_test_scaled)


model_l1 = LogisticRegressionCustom(learning_rate=0.01, n_iterations=1000, regularization='l1', lambda_reg=0.1)
model_l1.fit(X_train_selected, y_train)

y_pred_l1 = model_l1.predict(X_test_selected)

# Вычисление метрик
def evaluate_model(y_true, y_pred, model_name):
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
  recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
  print(f"Метрики для модели:")
  print(f"  Accuracy: {accuracy:.4f}")
  print(f"  Precision: {precision:.4f}")
  print(f"  Recall: {recall:.4f}")

In [ ]:
  Accuracy: 0.8641
  Precision: 0.8680
  Recall: 0.8641

Результат близкий, но он немного слабее, чем у библиотечной версии.

## Задача регрессии

Линейная регрессия:

In [ ]:
X = bl_rdata.drop(target_variable, axis=1)
y = bl_rdata[target_variable]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_lr = LinearRegression()
model_lr.fit(X_train_scaled, y_train)

y_pred_lr = model_lr.predict(X_test_scaled)


mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print(f"\nМетрики для регрессии (Linear Regression):")
print(f"  MAE: {mae_lr:.4f}")
print(f"  R-squared: {r2_lr:.4f}")

In [ ]:
Метрики для регрессии (Linear Regression):
  MAE: 0.2321
  R-squared: 0.6865

Результат давольно средний. Попробуем улучшить бейзлайн

## 3. Улучшение бейзлайна

Гипотезы: Признаки после One-Hot кодирования могут привносить шум, особенно если они не очень коррелируют с целевой переменной. Отбор наиболее релевантных признаков может помочь улучшить качество модели. Будем использовать SelectKBest с f_regression для выбора k лучших признаков и сравним результаты. Сформируем улучшенный бейзлайн по результатам проверки гипотез, а также обучим модели с улучшенным бейзлайном.

In [ ]:
selector = SelectKBest(score_func=f_regression, k=8)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

model_lr_selected = LinearRegression()
model_lr_selected.fit(X_train_selected, y_train)
y_pred_lr_selected = model_lr_selected.predict(X_test_selected)

mae_lr_selected = mean_absolute_error(y_test, y_pred_lr_selected)
r2_lr_selected = r2_score(y_test, y_pred_lr_selected)

print(f"\nМетрики для регрессии (Linear Regression) с отбором признаков:")
print(f"  MAE: {mae_lr_selected:.4f}")
print(f"  R-squared: {r2_lr_selected:.4f}")

In [ ]:
Метрики для регрессии (Linear Regression) с отбором признаков:
  MAE: 0.2221
  R-squared: 0.7079

Результат удалось незначительно улучшить.

## 4. Имплементация алгоритма машинного обучения

Модифицируем реализацию линейной регрессии:

In [ ]:
class LinearRegressionCustom:
    def __init__(self):
        self.w = None
        self.b = None

    def fit(self, X, y, learning_rate=0.01, n_iterations=1000):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(n_iterations):
            y_predicted = self.predict(X)
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.w = self.w - learning_rate * dw
            self.b = self.b - learning_rate * db

    def predict(self, X):
        return np.dot(X, self.w) + self.b

Обучаем имплементированные модели и оцением их качество

In [ ]:
X = bl_rdata.drop(target_variable, axis=1)
y = bl_rdata[target_variable]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_lr_custom = LinearRegressionCustom()
model_lr_custom.fit(X_train_scaled, y_train, learning_rate = 0.001, n_iterations = 2000) 

y_pred_lr_custom = model_lr_custom.predict(X_test_scaled)

mae_lr_custom = mean_absolute_error(y_test, y_pred_lr_custom)
r2_lr_custom = r2_score(y_test, y_pred_lr_custom)

print(f"\nМетрики для регрессии (Linear Regression Custom):")
print(f"  MAE: {mae_lr_custom:.4f}")
print(f"  R-squared: {r2_lr_custom:.4f}")

In [ ]:
MAE: 0.2502
R-squared: 0.6594

Как видно имплементрованная линейная регрессия работает немного хуже. Добавим техники из улучшенного бейзлайна

In [ ]:
selector = SelectKBest(score_func=f_regression, k=8)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

model_lr_custom = LinearRegressionCustom()
model_lr_custom.fit(X_train_selected, y_train, learning_rate = 0.001, n_iterations = 2000) # можно настроить

y_pred_lr_custom = model_lr_custom.predict(X_test_selected)

mae_lr_custom = mean_absolute_error(y_test, y_pred_lr_custom)
r2_lr_custom = r2_score(y_test, y_pred_lr_custom)

print(f"\nМетрики для регрессии (Linear Regression Custom):")
print(f"  MAE: {mae_lr_custom:.4f}")
print(f"  R-squared: {r2_lr_custom:.4f}")

In [ ]:
  MAE: 0.2362
  R-squared: 0.6854

Видно, что после улучшения бейзлайн всё ещё не выдаёт столь же хорошие результаты, что и улучшенная библиотечная реализация.